In [24]:
import glob, re
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
import collections


In [25]:
ZARR_PATH = "/home/aninotna/magister/tesis/justh2_pipeline/data/solar/solar_diario_grilla.zarr"  # referencia diaria (ghi)
CMIP6_DIR = "/home/aninotna/magister/tesis/justh2_pipeline/data/cmip6/historical"
CMIP6_GLOB = f"{CMIP6_DIR}/rsds_Amon_*historical*.nc"

In [26]:
# ---------- utils ----------
def to_lon180(lon):
    # Convierte 0..360 a -180..180 si es necesario
    lon2 = ((lon + 180) % 360) - 180
    # Ordena
    order = np.argsort(lon2)
    return lon2[order], order

In [27]:
def std_names(ds):
    rename = {}
    if "latitude" in ds.coords and "lat" not in ds.coords: rename["latitude"]="lat"
    if "longitude" in ds.coords and "lon" not in ds.coords: rename["longitude"]="lon"
    return ds.rename(rename) if rename else ds


In [28]:
def ensure_lat_ascending(ds):
    if "lat" in ds.coords and ds.lat.ndim==1 and ds.lat.values[0] > ds.lat.values[-1]:
        ds = ds.sortby("lat")
    return ds

In [29]:
def kge(sim, obs):
    m = ~np.isnan(sim) & ~np.isnan(obs)
    if m.sum() < 3: return np.nan
    r = np.corrcoef(sim[m], obs[m])[0,1]
    so, ss = np.std(obs[m]), np.std(sim[m])
    mo, ms = np.mean(obs[m]), np.mean(sim[m])
    if so==0 or mo==0: return np.nan
    alpha, beta = ss/so, ms/mo
    return 1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2)


In [30]:
def metrics(sim, obs):
    m = ~np.isnan(sim) & ~np.isnan(obs)
    if m.sum()==0: return dict(bias=np.nan, rmse=np.nan, r=np.nan, kge=np.nan, n=0)
    d = sim[m]-obs[m]
    return dict(
        bias=float(np.nanmean(d)),
        rmse=float(np.sqrt(np.nanmean(d**2))),
        r=float(np.corrcoef(sim[m], obs[m])[0,1]) if m.sum()>2 else np.nan,
        kge=float(kge(sim, obs)),
        n=int(m.sum()),
    )

In [31]:
def ym_int_from_time(tcoord):
    try:
        yrs = tcoord.dt.year.astype("int32"); mns = tcoord.dt.month.astype("int32")
        return (yrs*100 + mns).astype("int32").values
    except Exception:
        pass
    vals = np.asarray(tcoord.values)
    # datetime64
    if np.issubdtype(vals.dtype, np.datetime64):
        dt = pd.to_datetime(vals)
        return (dt.year*100 + dt.month).astype("int32")
    # cftime/objetos
    years  = np.fromiter((getattr(v, "year")  for v in vals), dtype=np.int32, count=len(vals))
    months = np.fromiter((getattr(v, "month") for v in vals), dtype=np.int32, count=len(vals))
    return years*100 + months

Se abre el dataset observado (irradiancia diaria ghi en W/m²).

Se estandarizan coordenadas (lat, lon) y se asegura orden ascendente.

In [32]:
# ---------------- Referencia mensual (zarr) ----------------
# Abrir zarr (usa dask por defecto)
ref = xr.open_zarr(ZARR_PATH)
ref = std_names(ref)
ref = ensure_lat_ascending(ref)


Se selecciona la variable de referencia: Global Horizontal Irradiance (ghi).

In [33]:
# Variable de referencia
ref_var = "ghi"
assert ref_var in ref.data_vars, f"'{ref_var}' no existe en {ZARR_PATH}"
ref_da = ref[ref_var]  # (date, lat, lon)


Si las longitudes están en 0–360, se convierten a -180–180.

In [34]:
# (Opcional) Si las lons vinieran 0–360, pásalas a −180–180
if ref_da.lon.max() > 180:
    lons, order = to_lon180(ref_da.lon.values)
    ref_da = ref_da.isel(lon=order).assign_coords(lon=lons)


Se pasa de diario → mensual (promedio mensual).

Se renombra date → time para que coincida con CMIP6.

Se asegura que las fechas estén redondeadas al día (evita problemas de horas).

In [35]:
# Diario -> Mensual (promedio mensual de W/m^2 diarios)
ref_mon = ref_da.resample(date="MS").mean("date")  # (time≈'date', lat, lon)
# Renombrar 'date' a 'time' para alinear con CMIP6
ref_mon = ref_mon.rename({"date":"time"})
ref_mon = ref_mon.assign_coords(time=ref_mon["time"].dt.floor("D"))


Se restringe al período común con CMIP6: 2004–2014.

In [36]:
# Años comunes esperados (2004–2014)
ref_mon = ref_mon.sel(time=slice("2004-01-01","2014-12-31"))

In [37]:
# ---------------- Loop modelos CMIP6 ----------------
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
files = sorted(glob.glob(CMIP6_GLOB))

Busca todos los archivos NetCDF de radiación superficial (rsds) en histórico.

Agrupa por modelo (porque un modelo puede venir en varios chunks).

In [38]:
# agrupa archivos por modelo
by_model = collections.defaultdict(list)
for f in files:  # 'files' ya venía del glob
    m = re.search(r"rsds_Amon_([^_]+)_historical", Path(f).name)
    model = m.group(1) if m else Path(f).name
    by_model[model].append(f)

rows = []
season_rows = []

Transforma la observación diaria a mensual y recorta al periodo común.

Abre cada modelo CMIP6, concatena, recorta y estandariza coords.

Alinea temporalmente observación y modelo.

Interpola espacialmente el modelo al grid observado.

Calcula métricas de ajuste globales y por estación.

Guarda resultados en rows (globales) y season_rows (estacionales).


In [39]:
pred = {}  # => guarda por modelo el campo mensual regrilleado/alineado (ym,lat,lon)
for model, flist in sorted(by_model.items()):
    try:
        # abre y concatena todos los tramos del modelo
        parts = []
        for f in sorted(flist):
            try:
                ds = xr.open_dataset(f, decode_times=time_coder, engine="netcdf4", chunks={})
            except Exception:
                ds = xr.open_dataset(f, decode_times=time_coder, chunks={})
            ds = std_names(ds); ds = ensure_lat_ascending(ds)
            var = "rsds" if "rsds" in ds.data_vars else list(ds.data_vars)[0]
            da  = ds[var]
            # pasa lon a -180..180 si hace falta
            if "lon" in da.coords and da.lon.max() > 180:
                lons, order = to_lon180(da.lon.values)
                da = da.isel(lon=order).assign_coords(lon=lons)
            parts.append(da)

        da_all = xr.concat(parts, dim="time").sortby("time")

        # recorte 2004–2014
        da_all = da_all.sel(time=slice("2004-01-01","2014-12-31"))
        if da_all.sizes.get("time", 0) == 0:
            print(f"[SKIP] {model}: sin meses en 2004–2014 (tras concatenar)")
            continue

        # alinear por AÑO-MES (YYYYMM)
        cmip_ym = ym_int_from_time(da_all["time"])
        ref_ym  = ym_int_from_time(ref_mon["time"])
        common_ym = np.intersect1d(cmip_ym, ref_ym)
        if common_ym.size == 0:
            print(f"[SKIP] {model}: sin meses comunes en 2004–2014 (tras concatenar)")
            continue

        da_aln  = da_all.assign_coords(ym=("time", cmip_ym)).swap_dims({"time":"ym"}).sel(ym=common_ym)
        ref_aln = ref_mon.assign_coords(ym=("time", ref_ym)).swap_dims({"time":"ym"}).sel(ym=common_ym)

        # interpola al grid de la referencia
        da_rg = da_aln.interp(lat=ref_aln.lat, lon=ref_aln.lon, method="linear").transpose("ym","lat","lon")
        pred[model] = da_rg 

        # métricas globales
        sim = da_rg.values.ravel(); obs = ref_aln.values.ravel()
        M = metrics(sim, obs)
        years_min = int(common_ym.min() // 100); years_max = int(common_ym.max() // 100)
        M.update(model=model, years_min=years_min, years_max=years_max, n=int((~np.isnan(sim)&~np.isnan(obs)).sum()))
        rows.append(M)
        print(f"OK {model} -> r={M['r']:.3f} RMSE={M['rmse']:.1f} bias={M['bias']:.1f} KGE={M['kge']:.3f} "
              f"({years_min}-{years_max}, n={M['n']})")

        # métricas por estación
        months = common_ym % 100
        # Etiquetas de estación (hemisferio sur)
        # DJF: 12,1,2  | MAM: 3,4,5  | JJA: 6,7,8  | SON: 9,10,11
        season_labels = np.empty_like(months, dtype=object)
        season_labels[np.isin(months, [12, 1, 2])] = "DJF"
        season_labels[np.isin(months, [3, 4, 5])]  = "MAM"
        season_labels[np.isin(months, [6, 7, 8])]  = "JJA"
        season_labels[np.isin(months, [9,10,11])]  = "SON"

        for s in ["DJF", "MAM", "JJA", "SON"]:
            idx = np.where(season_labels == s)[0]
            if idx.size == 0:
                continue
            # Selección por índice sobre la dimensión 'ym'
            sim_s = da_rg.isel(ym=idx).values.ravel()
            obs_s = ref_aln.isel(ym=idx).values.ravel()
            Ms = metrics(sim_s, obs_s)
            Ms.update(model=model, season=s)
            season_rows.append(Ms)

    except Exception as e:
        print(f"[ERROR] {model}: {e}")

OK ACCESS-CM2 -> r=0.955 RMSE=35.8 bias=18.0 KGE=0.880 (2004-2014, n=43560)
OK ACCESS-ESM1-5 -> r=0.961 RMSE=39.8 bias=27.2 KGE=0.857 (2004-2014, n=43560)
OK CAMS-CSM1-0 -> r=0.953 RMSE=47.8 bias=35.3 KGE=0.834 (2004-2014, n=43560)
OK GFDL-ESM4 -> r=0.956 RMSE=39.9 bias=25.7 KGE=0.859 (2004-2014, n=43560)
OK GISS-E2-1-G -> r=0.940 RMSE=38.2 bias=-12.0 KGE=0.844 (2004-2014, n=43560)
OK GISS-E2-1-G-CC -> r=0.937 RMSE=37.9 bias=-10.2 KGE=0.869 (2004-2014, n=43560)
OK GISS-E2-1-H -> r=0.935 RMSE=39.0 bias=-8.3 KGE=0.823 (2004-2014, n=43560)
OK IPSL-CM6A-LR -> r=0.930 RMSE=41.6 bias=-8.3 KGE=0.781 (2004-2014, n=43560)


In [40]:
df_annual = pd.DataFrame(rows)  # columnas: model, r, rmse, bias, kge, n, years_min, years_max
df_season = pd.DataFrame(season_rows)  # columnas: model, season, r, rmse, bias, kge, n

# Pivot KGE por estación
kge_season = df_season.pivot_table(index="model", columns="season", values="kge")

# Unimos anual + estaciones
score_df = df_annual.set_index("model")[["kge", "r", "rmse", "bias"]].join(kge_season)

# Calcula Score (usa NaN-safe: rellena con kge anual si falta)
for s in ["DJF","SON"]:
    if s not in score_df: score_df[s] = None
score_df["Score"] = 0.4*score_df["kge"] + 0.3*score_df["DJF"].fillna(score_df["kge"]) + 0.3*score_df["SON"].fillna(score_df["kge"])

# Ordena y toma top-3
top3 = score_df.sort_values("Score", ascending=False).head(3)
print(top3[["kge","DJF","SON","Score","r","rmse","bias"]].round(3))


                  kge    DJF    SON  Score      r    rmse    bias
model                                                            
ACCESS-CM2      0.880  0.515  0.778  0.740  0.955  35.770  18.039
GFDL-ESM4       0.859  0.525  0.734  0.721  0.956  39.924  25.678
GISS-E2-1-G-CC  0.869  0.507  0.734  0.720  0.937  37.933 -10.180


Calcula pesos por latitud = cos(lat) (porque las celdas más cercanas a los polos representan menos área).

Devuelve un “broadcaster”: se puede aplicar sobre un DataArray (lat, lon).

In [41]:
# === Helpers ===
def area_weights(lat):
    w = np.cos(np.deg2rad(lat.values))  # 1D
    return xr.DataArray(w, dims=["lat"]).broadcast_like  # retorna un "broadcaster"

Promedio ponderado en 2D (lat, lon).

Usa w para dar más peso a zonas de mayor área geográfica.

In [42]:
def wmean2D(da, w):
    # w: DataArray broadcastable a (lat,lon)
    num = (da * w(da)).sum(("lat","lon"), skipna=True)
    den = w(da).sum(("lat","lon"))
    return num / den

Aplana arrays y elimina NaN antes de compararlos.

In [43]:
def flatten_valid(a, b):
    m = np.isfinite(a) & np.isfinite(b)
    return a[m], b[m]

Calcula correlación espacial entre patrones medios anuales.

Responde: ¿el modelo reproduce bien el “mapa” anual de radiación?

In [44]:
def pcc_spatial_annual(sim, obs):
    # Promedio temporal para patrón espacial anual
    s = sim.mean("ym", skipna=True)
    o = obs.mean("ym", skipna=True)
    a, b = flatten_valid(s.values.ravel(), o.values.ravel())
    if a.size < 3: return np.nan
    return float(np.corrcoef(a,b)[0,1])

Promedia espacialmente → serie mensual ponderada.

Luego obtiene la climatología: promedio de todos los eneros, febreros, etc.

Devuelve un ciclo de 12 valores (1..12).

In [45]:
def annual_cycle_series(da, w):
    # Promedio espacial ponderado, luego climatología mensual (12)
    # ym es YYYYMM → extraemos el mes:
    months = (da["ym"].values % 100).astype(int)
    da_m = wmean2D(da, w)  # (ym)
    # Construye una DA con coord 'month'
    da_mc = xr.DataArray(da_m.values, coords={"month": ("ym", months)}, dims=["ym"])
    cyc = da_mc.groupby("month").mean("ym")  # 12 valores
    # reordena 1..12
    return cyc.sel(month=np.arange(1,13))

r: correlación de Pearson entre ciclos.

std_ratio: qué tanto varía el modelo respecto a la obs.

In [46]:


def taylor_metrics(sim_cyc, obs_cyc):
    a, b = flatten_valid(sim_cyc.values, obs_cyc.values)
    if a.size < 3: return dict(r=np.nan, std_ratio=np.nan)
    r = float(np.corrcoef(a,b)[0,1])
    std_ratio = float(np.std(a, ddof=1)/np.std(b, ddof=1)) if np.std(b, ddof=1)>0 else np.nan
    return dict(r=r, std_ratio=std_ratio)

Ajusta una función armónica simple:

𝑦 = 𝑐 + 𝑎 cos(𝜃) + 𝑏 sin(𝜃)

Devuelve:

amp → amplitud del ciclo anual.

peak_month → mes donde ocurre el máximo (fase).

Esto responde: ¿el modelo reproduce el mes de máxima radiación y su intensidad?

In [47]:
def fourier_first_harmonic(cyc):
    # Ajusta y ~ c + a*cos(θ) + b*sin(θ), θ=2π*m/12, m=1..12
    m = np.arange(1,13)
    y = cyc.values.astype(float)
    if np.any(~np.isfinite(y)): return dict(amp=np.nan, peak_month=np.nan)
    theta = 2*np.pi*m/12
    X = np.column_stack([np.ones_like(m), np.cos(theta), np.sin(theta)])  # [c, a, b]
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    c, a_, b_ = beta
    amp = float(np.hypot(a_, b_))  # amplitud
    # fase φ donde cos(θ-φ) máximo → φ = atan2(b_, a_)
    phi = np.arctan2(b_, a_)
    # mes pico (1..12): θ = φ -> m = φ * 12 / (2π)
    m_peak = (phi * 12 / (2*np.pi)) % 12
    m_peak = 12 if np.isclose(m_peak,0) else m_peak
    return dict(amp=amp, peak_month=float(m_peak))

MBE: error medio sesgado.

RMSE: error cuadrático medio.

NRMSE_mu: error normalizado por la media observada.

In [48]:
def mbe_rmse_nrmse(sim, obs, w):
    diff = sim - obs
    # promedio espacio-tiempo ponderado
    d_ws = wmean2D(diff, w)              # (ym)
    o_ws = wmean2D(obs,  w)              # (ym)
    rmse = float(np.sqrt(np.nanmean(d_ws.values**2)))
    mbe  = float(np.nanmean(d_ws.values))
    nrmse_mu = float(rmse / np.nanmean(o_ws.values)) if np.nanmean(o_ws.values)!=0 else np.nan
    return dict(MBE=mbe, RMSE=rmse, NRMSE_mu=nrmse_mu)

Calcula todas las métricas extra para un modelo:

PCC_spatial: correlación espacial promedio.

cycle_r: correlación del ciclo anual.

cycle_std_ratio: comparación de amplitud relativa del ciclo.

cycle_amp_obs / sim: amplitud observada vs simulada.

cycle_peak_month_obs: mes del máximo en la obs (para comparar con modelo).

MBE, RMSE, NRMSE_mu: métricas de error.

In [49]:
# === Wrapper para un modelo ===
def eval_model_extra(sim, obs):
    # sim/obs: DataArray (ym, lat, lon)
    w = area_weights(obs.lat)
    pcc = pcc_spatial_annual(sim, obs)

    sim_cyc = annual_cycle_series(sim, w)
    obs_cyc = annual_cycle_series(obs, w)
    tay = taylor_metrics(sim_cyc, obs_cyc)
    four = fourier_first_harmonic(obs_cyc) | {"amp_sim": fourier_first_harmonic(sim_cyc)["amp"]}

    err = mbe_rmse_nrmse(sim, obs, w)
    return {
        "PCC_spatial": pcc,
        "cycle_r": tay["r"],
        "cycle_std_ratio": tay["std_ratio"],
        "cycle_amp_obs": four["amp"],
        "cycle_amp_sim": four["amp_sim"],
        "cycle_peak_month_obs": four["peak_month"],
        "MBE": err["MBE"],
        "RMSE": err["RMSE"],
        "NRMSE_mu": err["NRMSE_mu"]
    }


Es lo que permite no solo decir “qué tan bien ajusta en promedio” (como KGE), sino también “qué tan bien captura la estacionalidad y los patrones espaciales”.

In [50]:
extra_rows = []
for model, da_sim in pred.items():
    Mx = eval_model_extra(da_sim, ref_aln)
    Mx["model"] = model
    extra_rows.append(Mx)

df_extra = pd.DataFrame(extra_rows).set_index("model")
print(df_extra.round(3))

                PCC_spatial  cycle_r  cycle_std_ratio  cycle_amp_obs  \
model                                                                  
ACCESS-CM2            0.265    0.998            1.148         116.86   
ACCESS-ESM1-5         0.387    0.998            1.147         116.86   
CAMS-CSM1-0           0.253    0.992            1.272         116.86   
GFDL-ESM4             0.581    0.996            1.157         116.86   
GISS-E2-1-G          -0.099    0.990            1.062         116.86   
GISS-E2-1-G-CC       -0.104    0.990            1.097         116.86   
GISS-E2-1-H          -0.060    0.995            1.033         116.86   
IPSL-CM6A-LR         -0.070    0.994            1.004         116.86   

                cycle_amp_sim  cycle_peak_month_obs     MBE    RMSE  NRMSE_mu  
model                                                                          
ACCESS-CM2            134.154                 0.424  14.882  23.314     0.125  
ACCESS-ESM1-5         133.964          

En la comparación entre simulaciones históricas CMIP6 y la serie observacional de irradiancia, los resultados mostraron diferencias sustanciales en el desempeño de los modelos. A nivel global (2004–2014), ACCESS-CM2 alcanzó el mayor ajuste integrado (KGE = 0.880, RMSE = 35.8 W/m², r = 0.955), lo que refleja una buena reproducción de la variabilidad temporal aunque con un sesgo positivo (+18 W/m²). De manera similar, GFDL-ESM4 presentó un desempeño consistente (KGE = 0.859, RMSE = 39.9 W/m²), destacando particularmente en la representación del patrón espacial anual (PCC = 0.581, el valor más alto entre los modelos). Por su parte, GISS-E2-1-G-CC logró un KGE competitivo (0.869) con un sesgo reducido y negativo (–10 W/m²), aunque con deficiencias en la correlación espacial (PCC < 0).

En cuanto al ciclo anual, todos los modelos alcanzaron correlaciones elevadas (r ≈ 0.99), lo que indica que reproducen adecuadamente la estacionalidad. No obstante, ACCESS-CM2 y GFDL-ESM4 tendieron a sobreestimar la amplitud (∼15 %), mientras que GISS-E2-1-G-CC mantuvo una amplitud más próxima a la observada. Considerando en conjunto las métricas globales, estacionales y espaciales, se seleccionaron ACCESS-CM2 y GFDL-ESM4 como los modelos de referencia principales, debido a su mayor consistencia en la reproducción de la variabilidad temporal y espacial, complementados por GISS-E2-1-G-CC como alternativa secundaria que corrige sesgos sistemáticos aunque con limitaciones en el patrón espacial.

ACCESS-CM2 — mejor KGE y menor RMSE global.

GFDL-ESM4 — mejor patrón espacial (PCC) y buen KGE.

GISS-E2-1-G-CC — KGE competitivo y sesgo cercano a 0 (ligeramente negativo), útil como contraste.